In [130]:
import os, re, pdb
import pandas as pd
%run ./service_time/service_time_id_override.py

WORKING_DIR_LOC = './service_time/'
CSV_FOLDER_LOC = 'cleaned_files/'
FILE_LOC = './baseballdatabank-2019.2/core/People.csv'

def get_service_time_df(path_str, file_str):
    service_time_df = pd.read_csv(path_str + file_str)
    year_series = pd.Series([int(file_str[0:4])] * service_time_df.shape[0])
    service_time_df['year'] = year_series
    service_time_df = service_time_df.set_index('Player')
    try:
        service_time_df = service_time_df.drop('Club', axis=1)
    except:
        pass
    return(service_time_df)

df_list = [get_service_time_df(FOLDER_LOC, file) for file in file_list]
service_time_df = pd.concat(df_list)
people_df = pd.read_csv(FILE_LOC)
people_df = people_df.set_index(people_df['nameLast'] + ',' + people_df['nameFirst'])

def get_playerID_from_df(index):
    try:
        player_id = people_df.loc[index, 'playerID']
    except:
        return(None)
    if not isinstance(player_id, str):
        final_game_dates = people_df.loc[index, 'finalGame']
        is_playing_list = [int(date[0:4]) > 2005 for date in final_game_dates]
        if sum(is_playing_list)>1: 
            return(service_time_overrides[index])
        player_id = people_df.loc[index, 'playerID'][is_playing_list][0]
    return(player_id)

file_list = os.listdir(WORKING_DIR_LOC+CSV_FOLDER_LOC)

service_time_df = service_time_df[service_time_df['service_time']>6]

names_to_match = list(map(lambda x: re.sub('([A-Z]{1}[.])([A-Z]{1}[.])', r"\1 \2", x), service_time_df.index.values))
names_to_match = list(map(lambda x: re.sub(' Jr.', '', x), names_to_match))
service_time_df['new_names'] = names_to_match
service_time_df = service_time_df.set_index(service_time_df['new_names'])

IDs = {name: [get_playerID_from_df(name)] for name in set(names_to_match)}
IDs_df = pd.DataFrame.from_dict(IDs, orient='index')
IDs_df.columns = ['playerID']
del service_time_df['new_names']
service_time_with_IDs_df = service_time_df.join(IDs_df)
service_time_with_IDs_df.to_csv(WORKING_DIR_LOC+'service_time.csv')

In [131]:
no_match = [key for key in IDs.keys() if IDs[key] == None]

print('{} names with multiple players'.format(len(two_players)))
print('{} names with no match'.format(len(no_match)))

print(two_players)
print(no_match)

0 names with multiple players
0 names with no match
[]
[]


In [134]:
service_time_with_IDs_df[service_time_with_IDs_df['playerID']==None]

,Pos,service_time,year,playerID
